# Weighted training - XGBoost, RandomForest etc ....
Since we have imbalanced dataset I am looking for way to build model using weighted training. I decided to build a notebook in which I compare different models using regular and weighted training.


**This notebook is "in progress" but I encourage you to follow the progress of the work.** 

<div class="alert alert-block alert-success">
<b>Notebook scope:</b>
    <ul>
        <li>Curret top LB submission analysis.</li>
        <li>XGBoost - baseline & weighted training comparition - I checked there is no difference between CatBoost and LightBoost (slight differences). You can find my notebok - Keras weighted training section below.</li>
        <li>LightGBM - out of the box + weighted training + probability callibration.</li>
        <li>RandomForest - out of the box & weighted.</li>
        <li>Logistic Regression - out of the box & weighted.</li>
        <li>Sampling Techniques - Under Sampling.</li>
        <li>Sampling Techniques - Over Sampling - Synthetic Minority Oversampling Technique(SMOTE).</li>
        <li>Result comparition and conlusion.</li>
    </ul>
</div>

You can find the rest of my notebooks with TPS-05 here:

- [Pytorch NN for tabular - step by step](https://www.kaggle.com/remekkinas/tps-5-pytorch-nn-for-tabular-step-by-step)
- [CNN (2D Convolution) for solving TPS-05](https://www.kaggle.com/remekkinas/cnn-2d-convolution-for-solving-tps-05)
- [SHAP + LGBM - looking for best features](https://www.kaggle.com/remekkinas/shap-lgbm-looking-for-best-features/)
- [HydraNet!! ... Keras Stacked Ensemble ..](https://www.kaggle.com/remekkinas/tps-5-hydranet-keras-stacked-ensemble)

### My personal opinion (summary and conclusions):
- in this competition the main challange is dealing with data sparsity (not class imbalance) - creating new features, dimention reduction etc.
- I tried many ways to deal with imbalance - weighted training / cost sensitive training (out of the box, custom weights), uder and over sampling, probability calibration. All methods did not  improve results (still looking for solution or your feedback - it is chance that I am wrong).
- out of the box algorithms predicts class_2 and class_3 but do not predict correctly class_1 and class_4 (probably due to data sparsity)

# Why this is important?

## **If you look at the world through pink glasses all the colors seem to be pink ...**

# Hypothesis
* Models on public leaderbord currently look thorough pink glasses

In [ ]:
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve, classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

import xgboost as xgb 
from lightgbm import LGBMClassifier

import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

RANDOM_STATE = 2021

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-may-2021/train.csv", index_col = 'id')
test = pd.read_csv("../input/tabular-playground-series-may-2021/test.csv", index_col = 'id')
#train = train[~train.drop('target', axis = 1).duplicated()]

X = pd.DataFrame(train.drop("target", axis = 1))

lencoder = LabelEncoder()
y = pd.DataFrame(lencoder.fit_transform(train['target']), columns=['target'])

# EXPERIMENT #1
# Lets look on class distribution in TPS-05 TRAINING dataset

In [ ]:
sns.countplot(x = 'target', data= y)

# Lets look on class distribution on TOP leaderboard submission 
I am not specifically mentioning which results. I took the first ones that are on the TOP list.

In [ ]:
top_submission = pd.read_csv("../input/tps05-sample-top-submission/tps-05-sample-submission-001.csv", index_col = 'id')

top_submission.describe().T.style.bar(subset=['mean'], color='#205ff2')\
                            .background_gradient(subset=['std'], cmap='Reds')\
                            .background_gradient(subset=['50%'], cmap='coolwarm')

- As we can see in TOP Leaderboard submission there is no Class_1 .... almost no Class_4 .... It could be potential problem but we have to examine it more in next sections.
- Distribution is almost the same comparing with training dataset.

## Conclusion - distribution is ..... similar ("pink color" biased)???? Looks good on public leaderboard ... but what if class distribution in remaining 50% of test dataset )private leaderboard) is different? Still we should see pink color?

# EXPERIMENT #2
# Lets make a prediction using XGBoost model and test them on train dataset
# SCORE ON PUBLIC LEADERBOARD -> 1.08684 (without optimization)


We will use subset (20%) of training data to see accuracy of model. We want to see how model trains on dataset and then compare results with validation dataset distribution. 

In [ ]:
# We define X_test for results comparision 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state= RANDOM_STATE)

In [ ]:
def training(model, X_train_oof, y_train_oof, weighted = False, b_type = True, ):
    test_preds = None
    test_oof_preds = None
    train_rmse = 0
    val_rmse = 0
    n_splits = 10
    
    skf = StratifiedKFold(n_splits = n_splits, shuffle = True,  random_state = 0)
    for fold, (tr_index , val_index) in enumerate(skf.split(X_train_oof.values , y_train_oof.values)):

        print(f"\nFold {fold + 1}")

        x_train_o, x_val_o = X_train_oof.iloc[tr_index] , X_train_oof.iloc[val_index]
        y_train_o, y_val_o = y_train_oof.iloc[tr_index] , y_train_oof.iloc[val_index]
        
        if weighted:
            weights_y = weights_df.iloc[tr_index]

        eval_set = [(x_val_o, y_val_o)]
        
        if b_type:
            if weighted:
                model.fit(x_train_o, y_train_o, eval_set = eval_set, verbose = 500, sample_weight = weights_y)
            else:
                model.fit(x_train_o, y_train_o, eval_set = eval_set, verbose = 500)
        
        else:
            model.fit(x_train_o, y_train_o)

        train_preds = model.predict(x_train_o)
        train_rmse += mean_squared_error(y_train_o ,train_preds , squared = False)
        print("\n- Training RMSE : " , mean_squared_error(y_train_o ,train_preds , squared = False))

        val_preds = model.predict(x_val_o)
        val_rmse += mean_squared_error(y_val_o , val_preds , squared = False)
        print("- Validation RMSE : " , mean_squared_error(y_val_o , val_preds , squared = False))
        print('---------------')

        if test_preds is None:
            test_preds = model.predict_proba(test.values)
            test_oof_preds = model.predict_proba(X_test.values)
        else:
            test_preds += model.predict_proba(test.values)
            test_oof_preds += model.predict_proba(X_test.values)

    print("\nAverage Training RMSE : " , train_rmse / n_splits)
    print("Average Validation RMSE : " , val_rmse / n_splits)

    test_preds /= n_splits
    test_oof_preds /= n_splits
    
    return test_preds, test_oof_preds

In [ ]:
xgb_model = xgb.XGBClassifier(tree_method='gpu_hist',  eval_metric='mlogloss')
xgb_preds_weighted, y_pred = training(xgb_model, X_train, y_train)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sorted_idx = xgb_model.feature_importances_.argsort()
plt.barh(X_train.columns[sorted_idx], xgb_model.feature_importances_[sorted_idx])
plt.xlabel("Xgboost Feature Importance")

In [ ]:
y_preds = np.argmax(y_pred, axis=1)
print(f'MSE Score: {mean_squared_error(y_test,y_preds)}\n')
print(classification_report(y_test, y_preds))

sns.heatmap(pd.DataFrame(confusion_matrix(y_test, y_preds)), annot=True, linewidths=.5, fmt="d")


## In test dataset (dataset separated from training dataset) we have .... 

In [ ]:
sns.countplot(x = 'target', data= y_test)

## But our model see .... through pink glasses ...

In [ ]:
sns.countplot(x = 'target', data= pd.DataFrame(y_preds, columns=['target']))

## Upppssss - distribution is ..... pink color - class_2 biased ... Is it not? Am I making a mistake?



# We can check this using LGBM as well

In [ ]:
params = { 
        'objective': 'multiclass', 
        'num_class' : 4, 
        'metric': 'multi_logloss' 
    } 

In [ ]:
lgbm_model =  LGBMClassifier(**params)
lgbm_preds_weighted, y_pred = training(lgbm_model, X_train, y_train)

In [ ]:
y_preds = np.argmax(y_pred, axis=1)
print(f'MSE Score: {mean_squared_error(y_test,y_preds)}\n')
print(classification_report(y_test, y_preds))

sns.heatmap(pd.DataFrame(confusion_matrix(y_test, y_preds)), annot=True, linewidths=.5, fmt="d")

Conclutions:
- The same here - we have problem with prediction - model can not predict class_1 and class_3 well
- There is no bigger difference between XGBoost and Light Boost - when optimized (on oryginal dataset) it could improve better results but ... this is in my opiniion marginal gain (model does not see classes correctly) 

# EXPERIMENT 3 - WEIGHTED TRAININGS

# Explore class weight

In [ ]:
# I do not use this one - this is only for testing

largest_class_weight_coef = max(y_train.target.value_counts().values)/y_train.shape[0]
print(largest_class_weight_coef)

def BalancedSampleWeights(y_train, class_weight_coef):
    classes = np.unique(y_train, axis = 0)
    classes.sort()
    class_samples = np.bincount(y_train)
    total_samples = class_samples.sum()
    n_classes = len(class_samples)
    weights = total_samples / (n_classes * class_samples * 1.0)
    class_weight_dict = {key : value for (key, value) in zip(classes, weights)}
    class_weight_dict[classes[1]] = class_weight_dict[classes[1]] * class_weight_coef
    sample_weights = [class_weight_dict[i] for i in y_train]
    return sample_weights

weight = BalancedSampleWeights(y_train.target, largest_class_weight_coef)
print(len(weight))
weights_df= pd.DataFrame(weight, columns=['weight'])

In [ ]:
# I use only this one ...

from sklearn.utils.class_weight import compute_sample_weight
weights_df= pd.DataFrame(compute_sample_weight("balanced", y_train.target), columns=['weight'])

# Some models eg. RF, LR require weights as dictionary
dict_weight = dict(enumerate(class_weight.compute_class_weight('balanced', np.unique(y_train.target), y_train.target).flatten(), 0))
print(dict_weight)

# 1. XGBoost training weighted training 

* I do not use any hyperparameter optimization - it will be done as a last step (after all methods development)
* The weighted training for XGBoost seems to be not working. I am looking for a way to solve the problem.

### 1A. XGBoost - COST SENSITIVE LEARNING



In [ ]:
xgb_model_weighted = xgb.XGBClassifier(tree_method='gpu_hist',  eval_metric='mlogloss')
xgb_preds_weighted, y_pred = training(xgb_model_weighted, X_train, y_train, weighted = True)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sorted_idx = xgb_model_weighted.feature_importances_.argsort()
plt.barh(X_train.columns[sorted_idx], xgb_model_weighted.feature_importances_[sorted_idx])
plt.xlabel("Xgboost Feature Importance")

In [ ]:
y_preds = np.argmax(y_pred, axis=1)
print(f'MSE Score: {mean_squared_error(y_test,y_preds)}\n')
print(classification_report(y_test, y_preds))

sns.heatmap(pd.DataFrame(confusion_matrix(y_test, y_preds)), annot=True, linewidths=.5, fmt="d")

In [ ]:
sns.countplot(x = 'target', data= pd.DataFrame(y_preds, columns=['target']))

## Upppssss - something streange has happened ... distribution is ..... not biased but way from source ... Is it not? Am I making a mistake?

### 1B. LightGBM - COST SENSITIVE LEARNING

LightGBM provides two ways - class_weight = 'balanced' || class_weight = custom_weights

In [ ]:
params_weighted = { 
        'objective': 'multiclass', 
        'num_class' : 4, 
        'metric': 'multi_logloss',
        'class_weight': dict_weight  # or 'balanced'
    } 

In [ ]:
lgbm_model_weighted =  LGBMClassifier(**params_weighted)
lgbm_preds_weighted, y_pred = training(lgbm_model_weighted, X_train, y_train)

In [ ]:
y_preds = np.argmax(y_pred, axis=1)
print(f'MSE Score: {mean_squared_error(y_test,y_preds)}\n')
print(classification_report(y_test, y_preds))

sns.heatmap(pd.DataFrame(confusion_matrix(y_test, y_preds)), annot=True, linewidths=.5, fmt="d")

As we can see .... still model does not performing well .... we can use probability calibration with isotonic regression or logistic regression.

In [ ]:
from sklearn.calibration import CalibratedClassifierCV

calibrated_clf = CalibratedClassifierCV(base_estimator = lgbm_model_weighted, cv=3, ensemble = False)
calibrated_clf.fit(X_train, y_train)

y_pred = calibrated_clf.predict_proba(X_test)

In [ ]:
y_preds = np.argmax(y_pred, axis=1)
print(f'MSE Score: {mean_squared_error(y_test,y_preds)}\n')
print(classification_report(y_test, y_preds))

sns.heatmap(pd.DataFrame(confusion_matrix(y_test, y_preds)), annot=True, linewidths=.5, fmt="d")

Uppppssss ..... we are in starting point  ...... Not good ....

# 2. Sampling Techniques - Under Sampling

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(random_state=0)

X_train_under, y_train_under = undersample.fit_resample(X_train, y_train)
sns.countplot(x = 'target', data= y_train_under)

# Other methods
# - The Condensed Nearest Neighbor Rule — CondensedNearestNeighbour
# - Two modifications of CNN — TomekLinks
# - One-Sided Selection — OneSidedSelection
# - Edited Nearest Neighbors — EditedNearestNeighbors
# - Neighborhood Cleaning Rule — NeighborhoodCleaningRule

In [ ]:
xgb_model = xgb.XGBClassifier(tree_method='gpu_hist',  eval_metric='mlogloss')
xgb_preds_weighted, y_pred = training(xgb_model, X_train_under, y_train_under)

In [ ]:
y_preds = np.argmax(y_pred, axis=1)
print(f'MSE Score: {mean_squared_error(y_test,y_preds)}\n')
print(classification_report(y_test, y_preds))

sns.heatmap(pd.DataFrame(confusion_matrix(y_test, y_preds)), annot=True, linewidths=.5, fmt="d")

In [ ]:
sns.countplot(x = 'target', data= pd.DataFrame(y_preds, columns=['target']))

# 3. Sampling Techniques - Over Sampling 

In [ ]:
from imblearn.over_sampling import SMOTE

X_train_over, y_train_over = SMOTE().fit_resample(X_train, y_train)
sns.countplot(x = 'target', data= y_train_over)

# Other methods
# - Borderline-SMOTE
# - Borderline-SMOTE SVM
# - Adaptive Synthetic Sampling (ADASYN)

In [ ]:
xgb_model = xgb.XGBClassifier(tree_method='gpu_hist',  eval_metric='mlogloss')
xgb_preds_weighted, y_pred = training(xgb_model, X_train_over, y_train_over)

In [ ]:
y_preds = np.argmax(y_pred, axis=1)
print(f'MSE Score: {mean_squared_error(y_test,y_preds)}\n')
print(classification_report(y_test, y_preds))

sns.heatmap(pd.DataFrame(confusion_matrix(y_test, y_preds)), annot=True, linewidths=.5, fmt="d")

In [ ]:
sns.countplot(x = 'target', data= pd.DataFrame(y_preds, columns=['target']))

In [ ]:
sns.countplot(x = 'target', data= y_test)

## SMOTE looks good (class distribution) but .... metrics are .... not good :) !!!

# 4. BONUS

## 4A. RandomForest

### A. DecisionTreeClassifier baseline

In [ ]:
rf_model = DecisionTreeClassifier()
rf_preds_weighted, y_pred = training(rf_model, X_train, y_train, False, False)

In [ ]:
y_preds = np.argmax(y_pred, axis=1)
print(f'MSE Score: {mean_squared_error(y_test,y_preds)}\n')
print(classification_report(y_test, y_preds))

sns.heatmap(pd.DataFrame(confusion_matrix(y_test, y_preds)), annot=True, linewidths=.5, fmt="d")

### B. DecisionTreeClassifier weighted training (use "balanced" param) 

In [ ]:
rf_model_weighted_balanced = DecisionTreeClassifier(class_weight = 'balanced') # you can use class_weight{“balanced”, “balanced_subsample”}, dict or list of dicts, default=None
rf_preds_weighted, y_pred = training(rf_model_weighted_balanced, X_train, y_train, False, False)

In [ ]:
y_preds = np.argmax(y_pred, axis=1)
print(f'MSE Score: {mean_squared_error(y_test,y_preds)}\n')
print(classification_report(y_test, y_preds))

sns.heatmap(pd.DataFrame(confusion_matrix(y_test, y_preds)), annot=True, linewidths=.5, fmt="d")

### C. DecisionTreeClassifier weighted training (use custom weight params) 

In [ ]:
rf_model_weighted = DecisionTreeClassifier(class_weight = dict_weight)
rf_preds_weighted, y_pred = training(rf_model_weighted, X_train, y_train, False, False)

In [ ]:
y_preds = np.argmax(y_pred, axis=1)
print(f'MSE Score: {mean_squared_error(y_test,y_preds)}\n')
print(classification_report(y_test, y_preds))

sns.heatmap(pd.DataFrame(confusion_matrix(y_test, y_preds)), annot=True, linewidths=.5, fmt="d")

## 4B. LogisticRegression

### A. LogisticRegression baseline

In [ ]:
lr_model = LogisticRegression()
lr_preds_weighted, y_pred = training(lr_model, X_train, y_train, False, False)

In [ ]:
y_preds = np.argmax(y_pred, axis=1)
print(f'MSE Score: {mean_squared_error(y_test,y_preds)}\n')
print(classification_report(y_test, y_preds))

sns.heatmap(pd.DataFrame(confusion_matrix(y_test, y_preds)), annot=True, linewidths=.5, fmt="d")

### B. LogisticRegression weighted training (use "balanced" param)

In [ ]:
lr_model_weighted = LogisticRegression(class_weight = "balanced")
lr_preds_weighted, y_pred = training(lr_model_weighted, X_train, y_train, False, False)

In [ ]:
y_preds = np.argmax(y_pred, axis=1)
print(f'MSE Score: {mean_squared_error(y_test,y_preds)}\n')
print(classification_report(y_test, y_preds))

sns.heatmap(pd.DataFrame(confusion_matrix(y_test, y_preds)), annot=True, linewidths=.5, fmt="d")

In [ ]:
#sub = pd.read_csv("/kaggle/input/tabular-playground-series-may-2021/sample_submission.csv")

#predictions_df = pd.DataFrame(test_preds, columns = ["Class_1", "Class_2", "Class_3", "Class_4"])
#predictions_df['id'] = sub['id']

In [ ]:
#predictions_df.to_csv("xgboost_weighted_submission.csv", index = False)